# Solving the Travelling Salesman problem using Simulated Annealing

In [5]:
import numpy as np
import math
import itertools
import random
import matplotlib.pyplot as plt
import scipy.stats as stats

As our benchmark dataset we will use the famous TSPLIB 95 [Euclidean Symmetrical Instances](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/). The solver expects that the uncompressed files are available in the folder `ALL_tsp`. We start by converting `.tsp` files into a distance matrix $A$, such that $a_{i, j}$ corresponds to the distance between cities $i$ and $j$. Since we are dealing with symmetrical instances, the matrix is also symmetric.

In [6]:
def dist_arr(file_path):
    x_coord = []
    y_coord = []

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            

            if line.startswith("EDGE_WEIGHT_TYPE"):
                if not line.endswith("EUC_2D"):
                    print("Error Incompatible Types")
                    break

            if line[0].isdigit():
                x_coord.append(float(line.split()[1]))  
                y_coord.append(float(line.split()[2]))

    dist = np.zeros((len(x_coord), len(x_coord)))

    for i in range(len(x_coord)):
        for j in range(len(x_coord)):
            dist[i, j] = math.dist([x_coord[i], y_coord[i]], [x_coord[j], y_coord[j]])

    return dist

In [7]:
dist = dist_arr("ALL_tsp/a280.tsp")
print(dist[0:9, 0:9])

[[ 0.         20.         24.08318916 32.984845   32.984845   42.75511665
  55.71355311 63.2455532  61.18823416]
 [20.          0.         18.43908891 34.17601498 42.52058325 50.47771786
  65.60487787 72.11102551 68.        ]
 [24.08318916 18.43908891  0.         16.1245155  27.78488798 33.9411255
  49.51767361 55.31726674 50.47771786]
 [32.984845   34.17601498 16.1245155   0.         16.         18.86796226
  34.40930107 39.59797975 34.40930107]
 [32.984845   42.52058325 27.78488798 16.          0.         10.
  23.32380758 30.46309242 28.28427125]
 [42.75511665 50.47771786 33.9411255  18.86796226 10.          0.
  15.62049935 21.63330765 18.43908891]
 [55.71355311 65.60487787 49.51767361 34.40930107 23.32380758 15.62049935
   0.          8.         11.3137085 ]
 [63.2455532  72.11102551 55.31726674 39.59797975 30.46309242 21.63330765
   8.          0.          8.        ]
 [61.18823416 68.         50.47771786 34.40930107 28.28427125 18.43908891
  11.3137085   8.          0.        ]]

As a utility function, we now define `full_dist` to calculate the total length of a given route. We encode it as a vector, such that the $i$-th element is the $i$-th city we visit on our tour.

In [8]:
def full_dist(dist, route):
    total = 0
    for i in range(len(route)):
        total += dist[route[i], route[(i+1) % len(route)]]

    return total

(c) Julian Hein